In [94]:
import numpy as np
import os
import glob
import pandas as pd 
import matplotlib.pyplot as plt

In [183]:
# Load the times related to the hourly data for each catchment grid cell 
times = np.load('LinDykeRainfall/wholeyear/times.npy', allow_pickle=True)

# Get a list of all the individual grid cell .npy files
npy_files = glob.glob('LinDykeRainfall/wholeyear/*.npy')
# get rid of times and the total
npy_files = npy_files[2:]

# Create an array to store the daily values for all the grid cells in the catchment
all_daily_sums = np. array([])

# Add the daily data for each grid cell
for file in npy_files:
    one_cell = np.load(file, allow_pickle = True)
    df = pd.DataFrame({'Dates': pd.to_datetime(times), 'Precipitation': one_cell})
    daily_sums = df.set_index('Dates').resample('D')['Precipitation'].sum()
    all_daily_sums = np.append(all_daily_sums, daily_sums)

In [99]:
# # Add the daily data for each grid cell
# for file in npy_files:
#     one_cell = np.load(file, allow_pickle = True)
#     df = pd.DataFrame({'Dates': pd.to_datetime(times), 'Precipitation': one_cell})
#     daily_sums = df.set_index('Dates').resample('D')['Precipitation'].sum()
#     print(np.mean(daily_sums))

In [184]:
# Find the mean daily precipitation values across the area covering the catchment 
results = {}
results["mean"] = np.mean(all_daily_sums)
results["25th Percentile"] = np.percentile(all_daily_sums, 25)
results["50th Percentile"] = np.percentile(all_daily_sums, 50)
results["75th Percentile"] = np.percentile(all_daily_sums, 75)
results["90th Percentile"] = np.percentile(all_daily_sums, 90)
results["95th Percentile"] = np.percentile(all_daily_sums, 95)
results["99th Percentile"] = np.percentile(all_daily_sums, 99)

In [185]:
results

{'mean': 1.7685922520743045,
 '25th Percentile': 0.0,
 '50th Percentile': 0.10000000149011612,
 '75th Percentile': 1.8000000715255737,
 '90th Percentile': 5.400000095367432,
 '95th Percentile': 8.600000381469727,
 '99th Percentile': 18.200000762939453}

In [182]:
start_of_event = datetime.datetime(2022, 1, 1, 0, 0)
def make_antecedent_conditions (statistic, days):
    dates = []
    for day in range(days,0,-1):
        dates.append(start_of_event - (oneday*day))
    df = pd.DataFrame({"Times":dates,
                      'Rainfall':results[statistic]})
    df.to_csv("LinDykeAntecedentConditions/lindyke_daily_antecedent_conditions_{}_{}days.csv".format(statistic, days),
                    header = False,index= False, date_format='%Y/%m/%d %H:%M')
    return df
    
rainfall = make_antecedent_conditions("50th Percentile",4)